In [ ]:
from ML_utils import ML_utils
from gensim.models import word2vec
import shutil
import os
import numpy as np
import pandas as pd
import re
import logging
import sys

# train wordvec

In [ ]:
file_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\merge_wiki.txt"
save_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\merge_wiki_tw.txt"
ML_utils.Opencc_file(file_path,save_path,replace_old=True)

In [ ]:
userdict_path = './jieba_dict/ptt_userdict.txt'
jieba_dict = 'jieba_dict/jieba_merge.txt'
dir_path = r"./word2vec_model"
ML_utils.CallF_DirFile_save(dir_path, ML_utils.Jieba_file_segmentation, 
                            replace_old=False,regular=True,file_filter_=None,
                            dict_path=jieba_dict,load_userdict_path=userdict_path,HMM=False)

In [ ]:
dir_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model"
save_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\model_wordvec"
ML_utils.Word2vec_train(file_path=None,dir_path=dir_path,save_path=save_path,model_min_count=1)

In [ ]:
#test
save_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\model_wordvec"
model = word2vec.Word2Vec.load(save_path)
model.wv.most_similar(positive='台灣',topn=20)

# word2vec add token

In [ ]:
#add token
model_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\model_wordvec"
save_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\model_wordvec1"
train_file_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\jieba_dict\token\token.txt"
ML_utils.add_word2vec_word(model_path,save_path,train_file_path=train_file_path)

In [ ]:
#test
save_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\model_wordvec1"
model = word2vec.Word2Vec.load(save_path)
model.wv.most_similar(positive='__EOS.Token__',topn=20)

# data add token

In [ ]:
data_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\data\merge_ptt.txt"
save_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\data\merge_ptt_token.txt"
ML_utils.data_add_token_SOS_EOS(data_path,save_path)

# data jieba segmentation

In [ ]:
data_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\data\merge_ptt_token.txt"
save_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\data\merge_ptt_token_jieba.txt"
load_userdict_path = './jieba_dict/ptt_userdict.txt'
dict_path = 'jieba_dict/jieba_merge.txt'
ML_utils.Jieba_file_segmentation(data_path,save_path,
                                 dict_path=dict_path,load_userdict_path=load_userdict_path,HMM=False)

# add loss_word to wordvec

In [ ]:
def Null_to_space(list_in):
    index = []
    for num in range(len(list_in)):        
        remove_num = 0
        if list_in[num] == "" or list_in[num] == "\u3000":
            index.append(num)
    for num in range(len(index)):
        remove_index = index[num]-remove_num
        list_in.pop(remove_index)
        remove_num = remove_num+1
    return list_in

def get_pair_vec(model,pairs,num):
    pair = pairs[num]
    X = pair[0]
    Y = pair[1]
    loss_word = []
    try:
        for word in X:
            model.wv.get_vector(word)
        for word in Y:
            model.wv.get_vector(word)
    except:
        loss_word.append(word)
    return loss_word

data_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\data\merge_ptt_token_jieba.txt"
pairs = []
with open(data_path,"r",encoding="utf-8") as f:
    i = 0
    for line in f:
        pair = line.split("\t")
        X = pair[0].strip(" ").split(" ")
        X = Null_to_space(X)
        Y = pair[1].strip("\n").strip(" ").split(" ")
        Y = Null_to_space(Y)
        pairs.append([X,Y])

model_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\model_wordvec1"
save_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\model_wordvec1"
model = word2vec.Word2Vec.load(model_path)
loss_word_set = set()
for num in range(len(pairs)):
    loss_word = get_pair_vec(model,pairs,num)
    loss_word_set.update(loss_word)

sentences = []   
for word in list(loss_word_set):
    sentences.append([word])
print(sentences)    
model.vocabulary.min_count = 1
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
model.build_vocab(sentences, update=True)#训练该行
model.train(sentences,total_examples= model.corpus_count,epochs= model.epochs)
model.save(save_path)
model.wv.most_similar(positive=sentences[0],topn=5)